In [ ]:
import yaml
from sqlalchemy import create_engine, text

# set up
with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("/home/ubuntu/work/therapeutic_accelerator/config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)

# Create engine to connect to database
engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [6]:
# Read header from CSV to create columns
from csv import DictReader

csv_path = '/home/ubuntu/work/backup/terms/assay_terms.csv'

with open(csv_path, 'r') as f:
    d_reader = DictReader(f)

    #get fieldnames from DictReader object and store in list
    headers = d_reader.fieldnames
    
headers

['', 'index', 'paperId', 'corpusId', 'abstract']

In [9]:
lines_number = 5
with open(csv_path) as input_file:
    head = [next(input_file) for _ in range(lines_number)]
head

[',index,paperId,corpusId,abstract\n',
 '0,,39063bcddbd9decd4c8f5c5d3e8392280ab046ad,52273203,\n',
 '1,,6ec7c156b4173ad7ca0dbc654da9267474644a41,23708908,"Constitutive JAK/STAT3 signaling contributes to disease progression in many lymphoproliferative disorders. Recent genetic analyses have revealed gain-of-function STAT3 mutations in lymphoid cancers leading to hyperactivation of STAT3, which may represent a potential therapeutic target. Using a functional reporter assay, we screened 306 compounds with selective activity against various target molecules to identify drugs capable of inhibiting the cellular activity of STAT3. Top hits were further validated with additional models including STAT3-mutated natural killer (NK)-cell leukemia/lymphoma cell lines and primary large granular lymphocytic (LGL) leukemia cells to assess their ability to inhibit STAT3 phosphorylation and STAT3 dependent cell viability. We identified JAK, mTOR, Hsp90 and CDK inhibitors as potent inhibitors of both WT 

Delete old table if necessary to replace with new table

sql = text(''' 
    DROP TABLE IF EXISTS abstracts;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

In [32]:
# Create Table in DB first before uploading
from sqlalchemy import MetaData, Table, Column, Integer, String

metadata_obj = MetaData()
table_name = 'abstracts'
# Create abstracts metadata
abstracts = Table(
    table_name,
    metadata_obj,
    Column("id", Integer, nullable = True), # has to be integer so that dask can partition the table
    Column("index", String, nullable=True),
    Column("paperId", String(50)),
    Column("corpusId", String, nullable=True),
    Column("abstract", String, nullable=True),
)

metadata_obj.create_all(engine)

Has to be run in the terminal because you have to supply the password
- `psql --host=database-1.cuaho2dof33c.us-east-1.rds.amazonaws.com \
--port=5432 --username=postgres --password \
--dbname=postgres`

In [26]:
# Upload CSV to DB
## Copy and paste the output below into the postgres terminal. 
## HEADER true means that the csv file contains the header. Number of columns in csv has to match exactly with DB columns
print(rf"\copy {table_name} FROM {csv_path} WITH (FORMAT CSV, HEADER true, DELIMITER ',');")

\copy abstracts FROM /home/ubuntu/work/attributes/papers_abstracts.csv WITH (FORMAT CSV, HEADER true, DELIMITER ',');


In [33]:
# Check if it worked
import pandas as pd
sql = text(f''' 
    SELECT * FROM {table_name} LIMIT 5;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

test = pd.DataFrame(query.fetchall())
test.head()

,id,index,paperId,corpusId,abstract
0,0,None,39063bcddbd9decd4c8f5c5d3e8392280ab046ad,52273203,None
1,1,None,6ec7c156b4173ad7ca0dbc654da9267474644a41,23708908,Constitutive JAK/STAT3 signaling contributes t...
2,2,None,c856627242a754d2d756b32843523e6d7a089148,13232625,Summary: The current work characterizes young ...
3,3,None,d4c9b2fa2b760b5cf90ce8635a7dede5b4cd58a2,73484844,Ionotropic glutamate receptors (iGluRs) mediat...
4,4,None,7f13abe2c82bf0c66ca423e905d8f5967c4517b1,229159752,Aim The current pandemic of coronavirus diseas...
